In [100]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [101]:
!pip install stable-baselines3[extra]
!pip install gym

In [102]:
import os # Neil added
import gym 
# import gymnasium as gym # Neil added
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [103]:
environment_name = "CartPole-v0"

In [104]:
env = gym.make(environment_name)

In [105]:
environment_name

'CartPole-v0'

In [109]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        import time
        time.sleep(.01)
        action = env.action_space.sample()
        # print(env.step(action));break
        n_state, reward, done, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:14.0
Episode:2 Score:11.0
Episode:3 Score:15.0
Episode:4 Score:14.0
Episode:5 Score:58.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [79]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [80]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-1.6158437e+00,  1.1570229e+38,  1.1335742e-02,  2.2054553e+38],
      dtype=float32)

# 3. Train an RL Model

In [81]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cuda device


In [82]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 784  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 504         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009191375 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00575     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.61        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0162     |
|    value_loss           | 47.1        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [83]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [84]:
model.save(PPO_path)

/home/nsambhu/anaconda3/envs/RLC2/lib/python3.7/site-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [85]:
del model

In [89]:
os.getcwd()

'/home/nsambhu/github/ReinforcementLearningCourse'

In [91]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [92]:
from stable_baselines3.common.evaluation import evaluate_policy

In [98]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [94]:
env.close()

# 5. Test Model

In [95]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.15706605,  0.7064944 ,  0.01706835, -0.5725405 ], dtype=float32)}]


In [96]:
env.close()

# 6. Viewing Logs in Tensorboard

In [97]:
training_log_path = os.path.join(log_path, 'PPO_3')

NameError: name 'log_path' is not defined

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()